In [1]:
# To clean enviroment variables
# %reset

# To clean an specific environment variable
%reset -f variable_name


Don't know how to reset  variable_name, please run `%reset?` for details


## Classical Solution

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

try:
    import cplex
    from cplex.exceptions import CplexError
except:
    print("Warning: Cplex not found.")
import math

from qiskit.utils import algorithm_globals
from qiskit_algorithms import SamplingVQE
from qiskit_algorithms.optimizers import SPSA
from qiskit.circuit.library import RealAmplitudes
from qiskit.primitives import Sampler

Clean data

In [3]:
df = pd.read_csv("uscities.csv")
columnsNeeded = ["city", "lat", "lng"]

coordenatesDf = df[columnsNeeded].iloc[:10]

# print(coordenatesDf["lat"][0])
print(coordenatesDf)

           city      lat       lng
0      New York  40.6943  -73.9249
1   Los Angeles  34.1139 -118.4068
2       Chicago  41.8373  -87.6862
3         Miami  25.7839  -80.2102
4        Dallas  32.7936  -96.7662
5  Philadelphia  40.0077  -75.1339
6       Houston  29.7863  -95.3889
7       Atlanta  33.7627  -84.4224
8    Washington  38.9047  -77.0163
9        Boston  42.3188  -71.0846


In [4]:
# Initialize instance values by calculate the squared Euclidean distances between a set of coordinates
#  represented in the dataframe.

def generate_instance(coordenatesDf):
    n = coordenatesDf.shape[0]
    
    xc = coordenatesDf["lat"]
    yc = coordenatesDf["lng"]
    instance = np.zeros([n, n])
    for ii in range(0, n):
        for jj in range(ii + 1, n):
            instance[ii, jj] = (xc[ii] - xc[jj]) ** 2 + (yc[ii] - yc[jj]) ** 2
            instance[jj, ii] = instance[ii, jj]
    return xc, yc, instance

In [5]:
# Initialize the problem by randomly generating the instance
lat, lng, instance = generate_instance(coordenatesDf)

print(lat, lng)

0    40.6943
1    34.1139
2    41.8373
3    25.7839
4    32.7936
5    40.0077
6    29.7863
7    33.7627
8    38.9047
9    42.3188
Name: lat, dtype: float64 0    -73.9249
1   -118.4068
2    -87.6862
3    -80.2102
4    -96.7662
5    -75.1339
6    -95.3889
7    -84.4224
8    -77.0163
9    -71.0846
Name: lng, dtype: float64


In [6]:
# Inicialization of variables

n = coordenatesDf.shape[0]  # number of nodes + depot (n+1)
K = 2  # number of vehicles

### Get the data

class Initializer:
    def __init__(self, n):
        self.n = n

    def generate_instance(self):

        n = self.n

        # np.random.seed(33)
        np.random.seed(1543)

        xc = (np.random.rand(n) - 0.5) * 10
        yc = (np.random.rand(n) - 0.5) * 10

        instance = np.zeros([n, n])
        for ii in range(0, n):
            for jj in range(ii + 1, n):
                instance[ii, jj] = (xc[ii] - xc[jj]) ** 2 + (yc[ii] - yc[jj]) ** 2
                instance[jj, ii] = instance[ii, jj]

        return xc, yc, instance

### Initialize the problem by randomly generating the instance

initializer = Initializer(n)
xc, yc, instance = initializer.generate_instance()

print(xc, yc)

In [7]:
print(instance)

[[0.00000000e+00 2.02194109e+03 1.90679827e+02 2.61825024e+02
  5.84146046e+02 1.93310056e+00 5.79687760e+02 1.58244585e+02
  1.27594221e+01 1.07063043e+01]
 [2.02194109e+03 0.00000000e+00 1.00340617e+03 1.52836915e+03
  4.70058760e+02 1.90728075e+03 5.48551842e+02 1.15506278e+03
  1.73612525e+03 2.30671100e+03]
 [1.90679827e+02 1.00340617e+03 0.00000000e+00 3.13602228e+02
  1.64234910e+02 1.60907671e+02 2.04558188e+02 7.58515556e+01
  1.22446909e+02 2.75844965e+02]
 [2.61825024e+02 1.52836915e+03 3.13602228e+02 0.00000000e+00
  3.23237030e+02 2.28085308e+02 2.46412139e+02 8.14038783e+01
  1.82356390e+02 3.56679493e+02]
 [5.84146046e+02 4.70058760e+02 1.64234910e+02 3.23237030e+02
  0.00000000e+00 5.19999642e+02 1.09408086e+01 1.53308553e+02
  4.27404093e+02 7.50274014e+02]
 [1.93310056e+00 1.90728075e+03 1.60907671e+02 2.28085308e+02
  5.19999642e+02 0.00000000e+00 5.14742043e+02 1.25276257e+02
  4.76003876e+00 2.17380137e+01]
 [5.79687760e+02 5.48551842e+02 2.04558188e+02 2.46412139e

In [8]:
# -------------------- Classical solution using IBM ILOG CPLEX --------------------

class ClassicalOptimizer:
    def __init__(self, instance, n, K):

        self.instance = instance
        self.n = n  # number of nodes
        self.K = K  # number of vehicles

    def compute_allowed_combinations(self):
        f = math.factorial
        return f(self.n) / f(self.K) / f(self.n - self.K)

    def cplex_solution(self):

        # refactoring
        instance = self.instance
        n = self.n
        K = self.K

        my_obj = list(instance.reshape(1, n**2)[0]) + [0.0 for x in range(0, n - 1)]
        my_ub = [1 for x in range(0, n**2 + n - 1)]
        my_lb = [0 for x in range(0, n**2)] + [0.1 for x in range(0, n - 1)]
        my_ctype = "".join(["I" for x in range(0, n**2)]) + "".join(
            ["C" for x in range(0, n - 1)]
        )

        my_rhs = (
            2 * ([K] + [1 for x in range(0, n - 1)])
            + [1 - 0.1 for x in range(0, (n - 1) ** 2 - (n - 1))]
            + [0 for x in range(0, n)]
        )
        my_sense = (
            "".join(["E" for x in range(0, 2 * n)])
            + "".join(["L" for x in range(0, (n - 1) ** 2 - (n - 1))])
            + "".join(["E" for x in range(0, n)])
        )

        try:
            my_prob = cplex.Cplex()
            self.populatebyrow(my_prob, my_obj, my_ub, my_lb, my_ctype, my_sense, my_rhs)

            my_prob.solve()

        except CplexError as exc:
            print(exc)
            return

        x = my_prob.solution.get_values()
        x = np.array(x)
        cost = my_prob.solution.get_objective_value()

        return x, cost

    def populatebyrow(self, prob, my_obj, my_ub, my_lb, my_ctype, my_sense, my_rhs):

        n = self.n

        prob.objective.set_sense(prob.objective.sense.minimize)
        prob.variables.add(obj=my_obj, lb=my_lb, ub=my_ub, types=my_ctype)

        prob.set_log_stream(None)
        prob.set_error_stream(None)
        prob.set_warning_stream(None)
        prob.set_results_stream(None)

        rows = []
        for ii in range(0, n):
            col = [x for x in range(0 + n * ii, n + n * ii)]
            coef = [1 for x in range(0, n)]
            rows.append([col, coef])

        for ii in range(0, n):
            col = [x for x in range(0 + ii, n**2, n)]
            coef = [1 for x in range(0, n)]

            rows.append([col, coef])

        # Sub-tour elimination constraints:
        for ii in range(0, n):
            for jj in range(0, n):
                if (ii != jj) and (ii * jj > 0):

                    col = [ii + (jj * n), n**2 + ii - 1, n**2 + jj - 1]
                    coef = [1, 1, -1]

                    rows.append([col, coef])

        for ii in range(0, n):
            col = [(ii) * (n + 1)]
            coef = [1]
            rows.append([col, coef])

        prob.linear_constraints.add(lin_expr=rows, senses=my_sense, rhs=my_rhs)


In [9]:
# Instantiate the classical optimizer class
classical_optimizer = ClassicalOptimizer(instance, n, K)

In [10]:
# Print number of feasible solutions
print("Number of feasible solutions = " + str(classical_optimizer.compute_allowed_combinations()))


Number of feasible solutions = 45.0


In [11]:
# Solve the problem in a classical fashion via CPLEX
x = None
z = None
try:
    x, classical_cost = classical_optimizer.cplex_solution()
    # Put the solution in the z variable
    z = [x[ii] for ii in range(n**2) if ii // n != ii % n]
    # Print the solution
    print(z)
except:
    print("CPLEX may be missing.")


[-0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 1.0, 1.0, -0.0, -0.0, -0.0, 1.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, 1.0, 0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 1.0, -0.0, -0.0, -0.0, 0.0, 1.0, 0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, 1.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 1.0, -0.0, -0.0, 0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]


# Visualize the solution
def visualize_solution(xc, yc, x, C, n, K, title_str):
    plt.figure()
    plt.scatter(xc, yc, s=200)
    for i in range(len(xc)):
        plt.annotate(i, (xc[i] + 0.15, yc[i]), size=16, color="r")
    plt.plot(xc[0], yc[0], "r*", ms=10)

    plt.grid()

    for ii in range(0, n**2):

        if x[ii] > 0:
            ix = ii // n
            iy = ii % n
            plt.arrow(
                xc[ix],
                yc[ix],
                xc[iy] - xc[ix],
                yc[iy] - yc[ix],
                length_includes_head=True,
                head_width=0.25,
            )

    plt.title(title_str + " cost = " + str(int(C * 100) / 100.0))
    plt.show()


if x is not None:
    visualize_solution(xc, yc, x, classical_cost, n, K, "Classical")

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[39.487660, -97.594333], zoom_start=0)
marker_icon1 = folium.Icon(color = "red")
for i in range(len(xc)):
    if (i == 0):
        folium.Marker(location=[xc[i], yc[i]], icon=marker_icon1).add_to(m)
    else:
        folium.Marker(location=[xc[i], yc[i]], className='marker-label').add_to(m)

for ii in range(0, n**2):
    if x[ii] > 0:
        ix = ii // n
        iy = ii % n
        
        folium.PolyLine([(xc[ix], yc[ix]), (xc[iy], yc[iy])], color="blue").add_to(m)

m


## Quantum Solution

In [88]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit_algorithms import VQE


In [94]:
class QuantumOptimizer:
    def __init__(self, instance, n, K):

        self.instance = instance
        self.n = n
        self.K = K

    def binary_representation(self, x_sol=0):

        instance = self.instance
        n = self.n
        K = self.K

        A = np.max(instance) * 100  # A parameter of cost function

        # Determine the weights w
        instance_vec = instance.reshape(n**2)
        w_list = [instance_vec[x] for x in range(n**2) if instance_vec[x] > 0]
        w = np.zeros(n * (n - 1))
        for ii in range(len(w_list)):
            w[ii] = w_list[ii]

        # Some variables I will use
        Id_n = np.eye(n)
        Im_n_1 = np.ones([n - 1, n - 1])
        Iv_n_1 = np.ones(n)
        Iv_n_1[0] = 0
        Iv_n = np.ones(n - 1)
        neg_Iv_n_1 = np.ones(n) - Iv_n_1

        v = np.zeros([n, n * (n - 1)])
        for ii in range(n):
            count = ii - 1
            for jj in range(n * (n - 1)):

                if jj // (n - 1) == ii:
                    count = ii

                if jj // (n - 1) != ii and jj % (n - 1) == count:
                    v[ii][jj] = 1.0

        vn = np.sum(v[1:], axis=0)

        # Q defines the interactions between variables
        Q = A * (np.kron(Id_n, Im_n_1) + np.dot(v.T, v))

        # g defines the contribution from the individual variables
        g = (
            w
            - 2 * A * (np.kron(Iv_n_1, Iv_n) + vn.T)
            - 2 * A * K * (np.kron(neg_Iv_n_1, Iv_n) + v[0].T)
        )

        # c is the constant offset
        c = 2 * A * (n - 1) + 2 * A * (K**2)

        try:
            max(x_sol)
            # Evaluates the cost distance from a binary representation of a path
            fun = (
                lambda x: np.dot(np.around(x), np.dot(Q, np.around(x)))
                + np.dot(g, np.around(x))
                + c
            )
            cost = fun(x_sol)
        except:
            cost = 0

        return Q, g, c, cost

    def construct_problem(self, Q, g, c) -> QuadraticProgram:
        qp = QuadraticProgram()
        for i in range(n * (n - 1)):
            qp.binary_var(str(i))
        qp.objective.quadratic = Q
        qp.objective.linear = g
        qp.objective.constant = c
        return qp

    def solve_problem(self, qp):
        algorithm_globals.random_seed = 10598
        vqe = SamplingVQE(sampler=Sampler(), optimizer=SPSA(), ansatz=RealAmplitudes())
        #optimizer = MinimumEigenOptimizer(min_eigen_solver=vqe)

        meo = MinimumEigenOptimizer(min_eigen_solver=NumPyMinimumEigensolver())
        #result = meo.solve(qp)


        #optimizer = VQE(vqe)
        #result = optimizer.solve(qp)
        #result = Nump
        # compute cost of the obtained result
        #_, _, _, level = self.binary_representation(x_sol=result.x)
        #return result.x#, level
        return 1

In [95]:
# Instantiate the quantum optimizer class with parameters:
quantum_optimizer = QuantumOptimizer(instance, n, K)

In [71]:
# Check if the binary representation is correct
try:
    if z is not None:
        Q, g, c, binary_cost = quantum_optimizer.binary_representation(x_sol=z)
        print("Binary cost:", binary_cost, "classical cost:", classical_cost)
        if np.abs(binary_cost - classical_cost) < 0.01:
            print("Binary formulation is correct")
        else:
            print("Error in the binary formulation")
    else:
        print("Could not verify the correctness, due to CPLEX solution being unavailable.")
        Q, g, c, binary_cost = quantum_optimizer.binary_representation()
        print("Binary cost:", binary_cost)
except NameError as e:
    print("Warning: Please run the cells above first.")
    print(e)

Binary cost: 1779.6944624604657 classical cost: 1779.6944624600005
Binary formulation is correct


In [72]:
qp = quantum_optimizer.construct_problem(Q, g, c)

In [93]:
#quantum_solution, quantum_cost = quantum_optimizer.solve_problem(qp)
quantum_solution = quantum_optimizer.solve_problem(qp)

print(quantum_solution)

TypeError: VQE.__init__() missing 2 required positional arguments: 'ansatz' and 'optimizer'

qiskit.algorithm.minimum_engensolvers.SamplingVQE